In [162]:
#!/usr/bin/python3

from xml.dom import minidom
from CLARIAH_CMDI.xml2dict.processor import CMDI # load, xmldom2dict
import json
from SchemaLOD import Schema
from config import default_crosswalks_location, crosswalks_location, cbs_default_crosswalks
from rdflib import Graph, URIRef, Literal, BNode, plugin, Namespace
from rdflib.serializer import Serializer
from rdflib.namespace import RDF, RDFS
from config import cmdifile, ROOT, DATAVERSE_ID, API_TOKEN
from Semaf import Semaf
from jGraph import jGraph
import sys
import requests
from datetime import datetime

sm = Semaf()
schema = Schema()
cmdifile = '/data/CLARIAH_CMDI/cmdi/easy-dataset:35803_easy-file:4297789_IPNV_208_publicinfo.cmdi'
file = '0b01e4108001f63a_MKB_2008-01-01.dsc'
file = '0b01e41080021937_AKT_1975-01-01.dsc'
cmdifile = "/home/tikhonov/ODISSEI/data/cbs_all/%s" % file
cwfile = "test-cmdi-crosswalks.csv"
cv_server = "https://skosmos.dev.odissei.nl" #/rest/v1/search?vocab=cbs&query="
s = sm.loadcmdi(cmdifile)

In [163]:
defaultvalue = schema.default_schema(cbs_default_crosswalks)
defaultvalue

{'producerName': 'Centraal Bureau voor Statistiek',
 'producerAbbreviation': 'CBS',
 'otherIdAgency': 'CBS Dataontwerp ID',
 'authorName': 'Centraal Bureau voor Statistiek',
 'datasetContactName': 'CBS',
 'datasetContactEmail': 'Microdata@cbs.nl',
 'subject': 'Social Sciences',
 'distributorName': 'Centraal Bureau voor Statistiek'}

In [164]:
schemaURL = 'https://raw.githubusercontent.com/IQSS/dataverse/develop/scripts/api/data/metadatablocks/citation.tsv'
schemapd = schema.load_metadata_schema(schemaURL, 'citation')
schema.to_graph('citation', filename='citation')
schema.metadataframe

b'Skipping line 72: expected 17 fields, saw 18\nSkipping line 85: expected 17 fields, saw 22\n'


Index(['#datasetField', 'name', 'title', 'description', 'watermark',
       ' fieldType', 'displayOrder', 'displayFormat', 'advancedSearchField',
       'allowControlledVocabulary', 'allowmultiples', 'facetable',
       'displayoncreate', 'required', 'parent', 'metadatablock_id', 'termURI'],
      dtype='object')


,#datasetField,name,title,description,watermark,fieldType,displayOrder,displayFormat,advancedSearchField,allowControlledVocabulary,allowmultiples,facetable,displayoncreate,required,parent,metadatablock_id,termURI
0,,title,Title,Full title by which the Dataset is known.,Enter title...,text,0.0,,True,False,False,False,True,True,,citation,http://purl.org/dc/terms/title
1,,subtitle,Subtitle,A secondary title used to amplify or state cer...,,text,1.0,,False,False,False,False,False,False,,citation,
2,,alternativeTitle,Alternative Title,A title by which the work is commonly referred...,,text,2.0,,False,False,False,False,False,False,,citation,http://purl.org/dc/terms/alternative
3,,alternativeURL,Alternative URL,"A URL where the dataset can be viewed, such as...","Enter full URL, starting with http://",url,3.0,"<a href=""#VALUE"" target=""_blank"">#VALUE</a>",False,False,False,False,False,False,,citation,https://schema.org/distribution
4,,otherId,Other ID,Another unique identifier that identifies this...,,none,4.0,:,False,False,True,False,False,False,,citation,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,,relatedDatasets,Related Datasets,"Any Datasets that are related to this Dataset,...",,textbox,72.0,,False,False,True,False,False,False,,citation,http://purl.org/dc/terms/relation
69,,otherReferences,Other References,Any references that would serve as background ...,,text,73.0,,False,False,True,False,False,False,,citation,http://purl.org/dc/terms/references
70,,originOfSources,Origin of Sources,"For historical materials, information about th...",,textbox,75.0,,False,False,False,False,False,False,,citation,
71,,characteristicOfSources,Characteristic of Sources Noted,Assessment of characteristics and source mater...,,textbox,76.0,,False,False,False,False,False,False,,citation,


In [165]:
defaultmetadata = {}
for i in schema.default.index:
    fieldname = str(schema.default.loc[i]['subfield'])
    print(fieldname)
    cfields = schema.Hierarchy(fieldname)
    print(cfields)
    if not cfields['fields']:
        # field without children
        triples = schema.Relations(fieldname, NESTED=True, relation='#exactMatch') 
        if triples:
            defaultmetadata[schema.get_object(triples[0])] = schema.default.loc[i]['value']
    else:
        metadatablock = {}
        for extrafield in cfields['fields']:
            triples = schema.Relations(extrafield, NESTED=True, relation='#altLabel') 
            print("\t %s => %s " % (extrafield, schema.get_object(triples[0])))
            if schema.get_object(triples[0]) in defaultvalue:
                vocnameS = schema.get_subject(triples[0])
                vocname = schema.vocURI(vocnameS)
                metadatablock[vocname] = defaultvalue[schema.get_object(triples[0])]
        defaultmetadata[schema.rootURI(schema.termURI(cfields['root']))] = metadatablock
        
        #print(cfields['root'])
        #print(schema.termURI(cfields['root']))        

producerName
{'root': 'https://dataverse.org/schema/citation/producer', 'fields': ['https://dataverse.org/schema/citation/producerName', 'https://dataverse.org/schema/citation/producerAffiliation', 'https://dataverse.org/schema/citation/producerAbbreviation', 'https://dataverse.org/schema/citation/producerURL', 'https://dataverse.org/schema/citation/producerLogoURL']}
	 https://dataverse.org/schema/citation/producerName => producerName 
	 https://dataverse.org/schema/citation/producerAffiliation => producerAffiliation 
	 https://dataverse.org/schema/citation/producerAbbreviation => producerAbbreviation 
	 https://dataverse.org/schema/citation/producerURL => producerURL 
	 https://dataverse.org/schema/citation/producerLogoURL => producerLogoURL 
producerAbbreviation
{'root': 'https://dataverse.org/schema/citation/producer', 'fields': ['https://dataverse.org/schema/citation/producerName', 'https://dataverse.org/schema/citation/producerAffiliation', 'https://dataverse.org/schema/citation/

In [166]:
print(json.dumps(defaultmetadata, indent=4))

{
    "https://dataverse.org/schema/citation/Producer": {
        "https://dataverse.org/schema/citation/producer#Name": "Centraal Bureau voor Statistiek",
        "https://dataverse.org/schema/citation/producer#Abbreviation": "CBS"
    },
    "https://dataverse.org/schema/citation/OtherID": {
        "https://dataverse.org/schema/citation/otherId#Agency": "CBS Dataontwerp ID"
    },
    "http://purl.org/dc/terms/creator": {
        "https://dataverse.org/schema/citation/author#Name": "Centraal Bureau voor Statistiek"
    },
    "https://dataverse.org/schema/citation/Contact": {
        "https://dataverse.org/schema/citation/datasetContact#Name": "CBS",
        "https://dataverse.org/schema/citation/datasetContact#E-mail": "Microdata@cbs.nl"
    },
    "http://purl.org/dc/terms/subject": "Social Sciences",
    "https://dataverse.org/schema/citation/Distributor": {
        "https://dataverse.org/schema/citation/distributor#Name": "Centraal Bureau voor Statistiek"
    }
}


In [167]:
crosswalks = schema.crosswalks(crosswalks_location)
crosswalks

{'/Dataontwerpversies/Versie/Dataontwerp/Beschrijving': 'dsDescriptionValue',
 '/Dataontwerpversies/Versie/Dataontwerp/EngelstaligePublicatietekst': 'dsDescriptionValue',
 '/Dataontwerpversies/Versie/Dataontwerp/Contextvariabelen/Contextvariabele/VerkorteSchrijfwijzeNaamVariabele': 'topicClassValue',
 '/Dataontwerpversies/OorspronkelijkeNaam': 'title',
 '/Dataontwerpversies/Versie/Dataontwerp/VerkorteSchrijfwijzeNaamDataontwerp': 'alternativeTitle',
 '/Dataontwerpversies/Versie/Dataontwerp/Id': 'otherIdValue',
 '/Dataontwerpversies/Versie/Dataontwerp/Trefwoorden/Trefwoord': 'keywordValue',
 '/Dataontwerpversies/Versie/Dataontwerp/Dataontwerpgroeppad"': 'keywordValue',
 '/Dataontwerpversies/Versie/Dataontwerp/Themas/Thema': 'topicClassValue',
 '/Dataontwerpversies/Versie/Dataontwerp/GeldigVanaf': 'publicationDate',
 '/Dataontwerpversies/Versie/Dataontwerp/SoortData': 'kindOfData',
 '/Dataontwerpversies/Versie/Dataontwerp/TypeVerslagperiode': 'frequencyOfDataCollection',
 '/Dataontwerpve

In [168]:
class GraphBuilder():
    def __init__(self, thisobject=None, RootRef=None, crosswolksfile=None, thisformat='json', graphformat=None, debug=False):
        self.stats = {}
        self.json = {}
        self.context = json.loads(thisobject)
        self.RootRef = RootRef
        self.dictcontent = []
        self.mappings = {}
        self.locator = {}
        self.namespaces = {}
        self.EnrichFlag = False
        self.crosswalks = {}
        # Default Graph 
        self.g = Graph()
        self.level = 0
        self.format = None
        self.crosswalks = None
        self.statements = []
        if self.format:
            self.format = graphformat

    def externalCV(self, cv_server, concept):
        return
        url = "%s/%s" % (cv_server, "/rest/v1/search?vocab=cbs&query=%s" % concept)
        response = requests.get(url)
        staID = BNode()
        EXISTS = False
        self.g.add((staID, self.skosxl['literalForm'], Literal(concept)))
        if response.status_code == 200:
            CVdata = json.loads(response.content.decode('utf-8'))['results']
            for cvobject in CVdata:
                self.g.add((staID, self.skos['note'], Literal('Skosmos concept')))
                for k, v in cvobject.items():
                    EXISTS = True
                    if k == 'prefLabel':
                        self.g.add((staID, self.skos['prefLabel'], Literal(v)))
                    if k == 'altLabel':
                        self.g.add((staID, self.skos['altLabel'], Literal(v)))
            if EXISTS:
                return staID
            else:
                return
        return

    def clearpath(self, xpath):
        return xpath.replace('#document', '')

    def set_crosswalks(self, cw):
        self.crosswalks = cw
           
    def SetRef(self, value):
        # Set references with loaded semantic mappings
        #if value:
        #    value = self.clearpath(value)
        #    if value:
        #        if str(value)[0] == '/':
        #            value = value.lstrip()

        if value in self.mappings:
            RefURL = self.mappings[value]
        else:
            RefURL = "%s%s" % (self.RootRef, value)
        self.crosswalks[RefURL] = value
        #self.mappings[value] = RefURL
        return RefURL
    
    def setNamespaces(self):
        # Define namespaces
        ns1 = Namespace("%s" % self.RootRef)
        self.g.bind('cmdi', ns1)
        ns2 = Namespace("%s/#" % self.RootRef)
        self.g.bind('cmdidoc', ns2)
        ns3 = Namespace("%s/Keyword#" % self.RootRef)
        self.g.bind('keywords', ns3)
        ns4 = Namespace("https://dataverse.org/schema/citation")
        self.g.bind('citation', ns4)
        ns5 = Namespace("https://dataverse.org/schema/")
        self.g.bind('schema', ns5)
        ns6 = Namespace("http://purl.org/dc/terms/")
        self.g.bind('dcterms', ns6)

        for nsname in self.namespaces:
            ns = Namespace(nsname)
            self.g.bind(self.namespaces[nsname], "%s/" % ns)    

    def load_crosswalks(self, crossfile):
        with open(crossfile, encoding='utf-8') as fh:
            content = fh.readlines()
            for line in content:
                mapline = line.split(',')
                self.mappings[mapline[0]] = mapline[1]
        return self.mappings
    
    def rotatelist(self, thislist, previous_element, xpathroot, DEBUG=None):
        # previous_element = parent key
        # k = key
        # v = value
        #self.level = self.level + 1
        self.level = 0 
        for keyID in range(0, len(thislist)):
            key = thislist[keyID]
            if type(key) is dict:
                complexstatements = {}
                staID = BNode()
                staIDlocal = BNode()
                for k, v in key.items():
                    #root="%s/%s" % (self.RootRef, previous_element)
                    root = self.SetRef(previous_element)
                    # vty xpathroot = "%s/%s" % (xpathroot, k)
                    #kRef = "%s/%s" % (self.RootRef, k)
                    self.dictcontent.append({"list": root, "xpath": xpathroot, self.SetRef(k): v, 'type': type(v), 'sort': keyID })
                    if type(v) is str:
                        complexstatements[URIRef(self.SetRef(k))] = v
                        staIDstr = BNode()
                        #self.g.add((staIDstr, URIRef(self.SetRef(k)), Literal(item)))                        
                        self.g.add((staIDstr, URIRef(self.SetRef(k)), Literal(v)))
                        self.g.add((staIDstr, URIRef(self.SetRef(k)), Literal(v)))
                        self.g.add((staIDstr, self.skosxl['hiddenLabel'], Literal(self.clearpath("%s/%s" % (xpathroot, k)))))
                        self.g.add((staIDstr, RDF.value, Literal(v)))
                        self.statements.append({'xpath': self.clearpath("%s/%s" % (xpathroot, k)), 'value': v })
                        
                        if self.format == 'rich':
                            self.g.add((staIDstr, self.skosxl['note'], Literal('internal statement')))
                            self.g.add((staIDstr, self.skosxl['literalForm'], Literal(k)))
                        
                        conceptgraph = self.externalCV(cv_server, v)
                        if conceptgraph:
                            self.g.add((staIDstr, self.skos['exactMatch'], conceptgraph))
                        self.g.add((staIDlocal, URIRef(self.SetRef(k)), staIDstr))                            

                    elif type(v) is list:
                        complexarray = []
                        for item in v:
                            self.level = self.level + 1
                            complexarray.append({ self.SetRef(k): item, URIRef("%s#Vocabulary" % self.SetRef(k)) : "url" })
                            
                            # Create and add a new statement in the graph
                            staIDar = BNode()
                            self.g.add((staIDar, URIRef(self.SetRef(k)), Literal(item)))
                            self.g.add((staIDar, self.skosxl['hiddenLabel'], Literal(self.clearpath("%s/%s" % (xpathroot, k)))))
                            # vty self.g.add((staIDar, self.skos['broader'], Literal(previous_element)))
                            self.g.add((staIDar, self.skos['broader'], URIRef(self.SetRef(previous_element))))
                            self.g.add((staIDar, self.skos['prefLabel'], Literal(k)))
                            self.g.add((staIDar, RDF.value, Literal(item)))
                            if self.format == 'rich':
                                self.g.add((staIDar, self.skos['note'], Literal(self.level)))
                                self.g.add((staIDar, self.skosxl['literalForm'], Literal(k)))
                                self.g.add((staIDar, self.skosxl['note'], Literal('cycle statement')))
                            
                            if self.EnrichFlag:
                                self.g.add((staIDar, URIRef("%s#Vocabulary" % self.SetRef(k)), Literal('vocabulary name')))
                                self.g.add((staIDar, URIRef("%s#VocabularyURL" % self.SetRef(k)), Literal("http link to concept URI for %s" % item)))
                            # Add statements from array
                            self.g.add((staIDlocal, URIRef(self.SetRef(k)), staIDar))
                        complexstatements[URIRef(self.SetRef(k))] = complexarray
                    if DEBUG:
                        print(complexstatements)
                self.g.add((URIRef(root), URIRef(self.SetRef(k)), staIDlocal))
        return
    
    def rotate(self, thisdict, previous_element, DEBUG=None):
        self.cmdiloc = {}

        skos = Namespace('http://www.w3.org/2004/02/skos/core#')
        self.g.bind('skos', skos)        
        skosxl = Namespace('http://www.w3.org/2008/05/skos-xl#')
        self.skos = Namespace('http://www.w3.org/2004/02/skos/core#')
        self.skosxl = Namespace('http://www.w3.org/2008/05/skos-xl#')
        self.g.bind('skosxl', skosxl)        
        
        if (isinstance(thisdict,list)):
            #root="%s/%s" % (self.RootRef, previous_element)
            root = self.SetRef(previous_element)
            #kRef = "%s/%s" % (self.RootRef, k)
            self.dictcontent.append({"list": root, self.SetRef(k): v })
            #print("%s" % root)
            self.g.add((URIRef(root), URIRef(self.SetRef(k)), Literal(v)))
            self.g.add((URIRef(root), skos['prefLabel'], Literal(root)))
            #self.g.add(((URIRef(root), skos['altLabel'], Literal(k)))            
            return

        for k,v in thisdict.items():
            if (isinstance(v,dict)):
                if previous_element:
                    fullXpath = "%s/%s" % (previous_element, k)
                else:
                    fullXpath = k
                self.namespaces[self.SetRef(previous_element)] = k.lower()
                # vty if DEBUG:
                #print("XPath %s [%s/%s]" % (fullXpath, previous_element, k))
                self.rotate(v, fullXpath)
                ###self.rotate(v, k)
                #root="%s%s" % (self.RootRef, previous_element)
                root = self.SetRef(previous_element)
                #kRef = "%s/%s" % (self.RootRef, k)
                staID = BNode()
                staID = URIRef(self.RootRef)
                self.g.add((staID, URIRef(root), URIRef(self.SetRef(k))))
                #self.g.add((staID, skos['broader'], URIRef(nodename)))
                self.g.add((staID, skos['hiddenLabel'], Literal(self.clearpath(fullXpath))))
                self.g.add((staID, skos['altLabel'], Literal(k)))
                self.g.add((staID, RDF.value, Literal(v)))
                if self.format == 'rich':
                    self.g.add((staID, skos['note'], Literal('simple statement')))
                self.locator[root] = staID
                continue
            else:
                if (isinstance(v,list)):
                    if DEBUG:
                        print(k)
                    xpathroot = "%s/%s" % (previous_element, k)
                    self.rotatelist(v, k, xpathroot)
                    continue
                #root="%s%s" % (self.RootRef, previous_element)
                root = self.SetRef(previous_element)
                xpathroot = "%s/%s" % (previous_element, k)
                if DEBUG:
                    print(self.cmdiloc)
                #kRef = "%s/%s" % (self.RootRef, k)

                if self.SetRef(k) in self.cmdiloc:
                    try:
                        cache = self.cmdiloc['root']
                    except: 
                        cache = []

                    if type(cache) is list:
                        cache.append( { self.SetRef(k): v })
                    else:
                        cache = { self.SetRef(k): v }
                else:
                    self.cmdiloc = { self.SetRef(k): v }
                self.dictcontent.append({"parent": root, "xpath": xpathroot, self.SetRef(k): v, 'type': type(v) })

                # Link root and parents
                self.g.add((URIRef(root), skos['narrower'], URIRef(self.SetRef(k))))  

                # Add statement
                staID = BNode()
                self.locator[URIRef(self.SetRef(k))] = staID
                ### outdated self.g.add((URIRef(root), URIRef(self.SetRef(k)), Literal(v)))  
                #self.g.add((staID, skos['note1'], Literal(k)))
                self.g.add((URIRef(root), skosxl['hiddenLabel'], Literal(self.clearpath(xpathroot))))                
                self.g.add((URIRef(root), skos['broader'], URIRef(root)))
                if self.format == 'rich':
                    self.g.add((URIRef(root), skosxl['literalForm'], Literal(k)))
                    self.g.add((URIRef(root), skos['note'], Literal('parent statement')))
                                
                #self.g.add((URIRef(root), skosxl['Label'], Literal(k)))
                self.g.add((staID, skos['literalForm'], Literal(k)))
                self.g.add((staID, skosxl['hiddenLabel'], Literal(self.clearpath("%s" % (previous_element)))))
                self.g.add((staID, URIRef(self.SetRef(k)), Literal(v)))
                self.g.add((staID, RDF.value, Literal(v)))
                if self.format == 'rich':
                    self.g.add((staID, skos['note'], Literal('compound statement')))
                self.g.add((staID, skos['broader'], URIRef(root)))
                
                #self.g.add((URIRef(root), skosxl['LabelRelation'], staID))
                self.g.add((URIRef(self.SetRef(k)), skosxl['LabelRelation'], staID))
                self.g.add((URIRef(self.SetRef(k)), skosxl['hiddenLabel'], Literal(self.clearpath(xpathroot))))  ### ???
                self.g.add((URIRef(self.SetRef(k)), skos['broader'], URIRef(root)))            
                if self.format == 'rich':
                    self.g.add((URIRef(self.SetRef(k)), skosxl['literalForm'], Literal(k)))  
                    self.g.add((URIRef(self.SetRef(k)), skos['note'], Literal('compound statements container')))                  

        self.setNamespaces()
        return self.dictcontent

    def dataverse_export(self, data, savedmetadata=None):        
        metadata = {}
        if savedmetadata:
            metadata = savedmetadata
        for field, thisvalue in data.items():
            if field in self.crosswalks:       
                thisfield = crosswalks[field] 
                nested = schema.Hierarchy(thisfield)
                print(thisfield)
                print(nested)
                if 'root' in nested:
                    print(nested)
                    metadatablock = {}
                    valuefield = "%sValue" % thisfield
                    for field in nested['fields']:                
                        #if field in newfields:
                        print("\t\t %s %s" % (field, schema.vocURI(field)))
                        if schema.termURI(field) == schema.termURI(thisfield) or 'Value' in field: 
                            #schema.termURI(field) == schema.termURI(valuefield):
                            metadatablock[schema.vocURI(field)] = thisvalue
                    print(metadatablock)
                    metadata[schema.rootURI(nested['root'])] = metadatablock
                else:
                    termURI = schema.termURI(thisfield)
                    if termURI:
                        print("\t %s" % termURI)
                        metadata[termURI] = thisvalue
        return metadata        



In [169]:
cmdigraph = GraphBuilder(sm.json, "https://dataverse.org/schema/cbs/", graphformat='rich')
#cmdigraph.load_crosswalks(cwfile)
cmdigraph.set_crosswalks(crosswalks)
items = cmdigraph.rotate(cmdigraph.context, False)
cmdigraph.context

{'#document': {'Dataontwerpversies': {'OorspronkelijkeNaam': 'Arbeidskrachtentelling',
   'Versie': {'Dataontwerp': {'Id': '0b01e41080021937',
     'Dataontwerpgroeppad': 'Sociaaleconomische en ruimtelijke statistieken/Arbeid en lonen/Werkgelegenheid en lonen/Arbeidskrachtentelling',
     'Beschrijving': 'Arbeidskrachtentelling 1973',
     'VerkorteSchrijfwijzeNaamDataontwerp': 'AKT',
     'BeschrijvingVanDePopulatie': 'Personen',
     'TypeVerslagperiode': 'Twee jaar',
     'SoortData': 'Micro',
     'GeldigVanaf': '1973-01-01',
     'Trefwoorden': {'Trefwoord': ['Beroepsbevolking', 'Arbeidskracht']},
     'Themas': {'Thema': 'Dossiers; Historische reeksen'},
     'Contextvariabelen': {'Contextvariabele': [{'Variabele': {'Id': '0b01e41080209d29',
         'UniekeNaam': 'Plaats van een persoon in een huishouden',
         'Definitie': 'Plaats die een persoon in een huishouden inneemt als referentiepersoon, of ten opzichte van de referentiepersoon van een huishouden.',
         'GeldigV

In [176]:
cmdigraph.dictcontent

[{'parent': 'https://dataverse.org/schema/cbs/#document/Dataontwerpversies',
  'xpath': '#document/Dataontwerpversies/OorspronkelijkeNaam',
  'https://dataverse.org/schema/cbs/OorspronkelijkeNaam': 'Arbeidskrachtentelling',
  'type': str},
 {'parent': 'https://dataverse.org/schema/cbs/#document/Dataontwerpversies/Versie/Dataontwerp',
  'xpath': '#document/Dataontwerpversies/Versie/Dataontwerp/Id',
  'https://dataverse.org/schema/cbs/Id': '0b01e41080021937',
  'type': str},
 {'parent': 'https://dataverse.org/schema/cbs/#document/Dataontwerpversies/Versie/Dataontwerp',
  'xpath': '#document/Dataontwerpversies/Versie/Dataontwerp/Dataontwerpgroeppad',
  'https://dataverse.org/schema/cbs/Dataontwerpgroeppad': 'Sociaaleconomische en ruimtelijke statistieken/Arbeid en lonen/Werkgelegenheid en lonen/Arbeidskrachtentelling',
  'type': str},
 {'parent': 'https://dataverse.org/schema/cbs/#document/Dataontwerpversies/Versie/Dataontwerp',
  'xpath': '#document/Dataontwerpversies/Versie/Dataontwerp/

In [177]:
concept = 'INV_WA'
cmdigraph.externalCV(cv_server, concept)

In [178]:
outputfile = './tests/cmdi_test'
cmdigraph.g.serialize(format='n3', destination="%s.nt" % outputfile)

<Graph identifier=N1890beb37caa4bca82bb3f02f527ca13 (<class 'rdflib.graph.Graph'>)>

In [179]:
s = Schema(cmdigraph.g)
field = 'https://dataverse.org/schema/citation/keyword#Term'
#triples = s.Relations(pred, NESTED=True, relation='#hiddenLabel')
#triples

In [180]:
pred = 'https://dataverse.org/schema/citation/keyword#Term'
pred = '#document/CMD/Components/OralHistoryInterviewDANS/Interviewer/Actor/Age'
pred = '#document/CMD/Components/OralHistoryInterviewDANS/InterviewGeneral/Access/Contact'
pred = '/Dataontwerpversies/OorspronkelijkeNaam'
#pred = '/Dataontwerpversies/Versie/Dataontwerp/Contextvariabelen/Contextvariabele/VerkorteSchrijfwijzeNaamVariabele'
print(cmdigraph.clearpath(pred))
#pred = '#document/Dataontwerpversies/Versie/Dataontwerp/Id'
x = cmdigraph.g
#x = cmdigraph.g.triples((Literal(pred), None, None))
#x = cmdigraph.g.triples((None, cmdigraph.skosxl['hiddenLabel'], Literal(pred)))
x = cmdigraph.g.triples((None, RDF.value, None))
#x = cmdigraph.g.triples((None, cmdigraph.skosxl['hiddenLabel'], None)) #Literal('Actor')))
metadata = defaultmetadata
#nested = schema.Hierarchy(thisfield)

for s, p, o in x:
    print("\n")
    print("%s %s %s" % (s, p, o))
    y = cmdigraph.g.triples((s, None, None))
    if y:
        for s1, p1, o1 in y:
            print("\t%s %s %s" % (s1, p1, o1))
            t = 1
            y2 = cmdigraph.g.triples((o1, cmdigraph.skosxl['hiddenLabel'], None))
            for s2, p2, o2 in y2:
                print("\t\t%s %s %s" % (s2, p2, o2))
                y3 = cmdigraph.g.triples((o2, None, None))
                for s3, p3, o3 in y3:
                    #print("\t\t\t%s %s %s" % (s3, p3, o3))
                    x = 1



/Dataontwerpversies/OorspronkelijkeNaam


Ncafae7240dc34af388461ce571fc8bfe http://www.w3.org/1999/02/22-rdf-syntax-ns#value Arbeidskrachtentelling
	Ncafae7240dc34af388461ce571fc8bfe http://www.w3.org/2004/02/skos/core#literalForm OorspronkelijkeNaam
	Ncafae7240dc34af388461ce571fc8bfe http://www.w3.org/2008/05/skos-xl#hiddenLabel /Dataontwerpversies
	Ncafae7240dc34af388461ce571fc8bfe https://dataverse.org/schema/cbs/OorspronkelijkeNaam Arbeidskrachtentelling
	Ncafae7240dc34af388461ce571fc8bfe http://www.w3.org/1999/02/22-rdf-syntax-ns#value Arbeidskrachtentelling
	Ncafae7240dc34af388461ce571fc8bfe http://www.w3.org/2004/02/skos/core#broader https://dataverse.org/schema/cbs/#document/Dataontwerpversies
		https://dataverse.org/schema/cbs/#document/Dataontwerpversies http://www.w3.org/2008/05/skos-xl#hiddenLabel /Dataontwerpversies/OorspronkelijkeNaam


Nf32d359ed9c9493495c5416e9c7d8342 http://www.w3.org/1999/02/22-rdf-syntax-ns#value 0b01e41080021937
	Nf32d359ed9c9493495c5416e9c7d8342 ht

In [181]:
    field = '/Dataontwerpversies/OorspronkelijkeNaam'
    field = '/Dataontwerpversies/Versie/Dataontwerp/Contextvariabelen/Contextvariabele/VerkorteSchrijfwijzeNaamVariabele'
    thisvalue = 'testtitle'
    data = {}
    data[field] = thisvalue
    data['/Dataontwerpversies/OorspronkelijkeNaam'] = 'title'
    data['/Dataontwerpversies/Versie/Dataontwerp/EngelstaligePublicatietekst'] = 'testdescription'
    metadata = cmdigraph.dataverse_export(data, metadata)
    print(json.dumps(metadata, indent=4))               

topicClassValue
{'root': 'https://dataverse.org/schema/citation/topicClassification', 'fields': ['https://dataverse.org/schema/citation/topicClassValue', 'https://dataverse.org/schema/citation/topicClassVocab', 'https://dataverse.org/schema/citation/topicClassVocabURI']}
{'root': 'https://dataverse.org/schema/citation/topicClassification', 'fields': ['https://dataverse.org/schema/citation/topicClassValue', 'https://dataverse.org/schema/citation/topicClassVocab', 'https://dataverse.org/schema/citation/topicClassVocabURI']}
		 https://dataverse.org/schema/citation/topicClassValue https://dataverse.org/schema/citation/topicClassification#Term
		 https://dataverse.org/schema/citation/topicClassVocab https://dataverse.org/schema/citation/topicClassification#Vocabulary
		 https://dataverse.org/schema/citation/topicClassVocabURI https://dataverse.org/schema/citation/topicClassification#Vocabulary URL
{'https://dataverse.org/schema/citation/topicClassification#Term': 'testtitle'}
title
{'field